# Deep Neural Net for Image Classification: Application

In [21]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from utils import *

In [22]:
#some random shit I even don't understand
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# importing dataset
def loadDataset():
    trainDataset = h5py.File("../../Week 2/datasets/train_catvnoncat.h5", "r")
    trainDatasetX = np.array(trainDataset["train_set_x"][:])
    trainDatasetY = np.array(trainDataset["train_set_y"][:])
    
    testDataset = h5py.File("../../Week 2/datasets/test_catvnoncat.h5")
    testDatasetX = np.array(testDataset["test_set_x"][:])
    testDatasetY = np.array(testDataset["test_set_y"][:])

    classes = np.array(trainDataset["list_classes"][:])

    trainDatasetY = trainDatasetY.reshape((1, trainDatasetY.shape[0]))
    testDatasetY = testDatasetY.reshape((1, testDatasetY.shape[0]))
    return trainDatasetX, trainDatasetY, testDatasetX, testDatasetY, classes

In [24]:
trainDatasetX, trainDatasetY, testDatasetX, testDatasetY, classes = loadDataset()
print(trainDatasetX.shape, trainDatasetY.shape)

(209, 64, 64, 3) (1, 209)


In [25]:
mTrain = trainDatasetX.shape[0]
numPx = trainDatasetX.shape[1]
mTest = testDatasetX.shape[0]

print(f"Number of training examples: {mTrain}")
print(f"Number of px in images: {numPx}")
print(f"Number of training examples: {mTest}")

Number of training examples: 209
Number of px in images: 64
Number of training examples: 50


In [26]:
trainXFlatten = trainDatasetX.reshape((trainDatasetX.shape[0], -1)).T
testXFlatten = testDatasetX.reshape((testDatasetX.shape[0], -1)).T

trainX = trainXFlatten / 255
testX = testXFlatten / 255

In [27]:
print(f"Shape trainX: {trainX.shape}")
print(f"Shape trainDatasetY: {trainDatasetY.shape}")
print(f"Shape testX: {testX.shape}")
print(f"Shape testDatasetY: {testDatasetY.shape}")


Shape trainX: (12288, 209)
Shape trainDatasetY: (1, 209)
Shape testX: (12288, 50)
Shape testDatasetY: (1, 50)


In [28]:
nx = trainX.shape[0]
nh = 7
ny = 1
layerDims = (nx, nh, ny)

In [29]:
def twoLayerModel(x, y, layerDims, learningRate = 0.0075, numIterations = 2500, printCost = False):
    np.random.seed(1)
    grads = {}
    costs = []
    m = x.shape[1]
    nx, nh, ny = layerDims
    params = initializeParams(nx, nh, ny)

    for i in range(0, numIterations):

        # print(type(x), type(params["w1"]), type(params["b1"]))
        # print(x.shape, params["w1"].shape, params["b1"].shape)

        a1, cache1 = linearActivationForward(x, params["w1"], params["b1"], "relu")
        a2, cache2 = linearActivationForward(a1, params["w2"], params["b2"], "sigmoid")

        cost = computeCost(a2, y)

        da2 = - (np.divide(y, a2) - np.divide(1-y, 1-a2))

        da1, dw2, db2 = linearActivationBackward(da2, cache2, "sigmoid")
        da0, dw1, db1, = linearActivationBackward(da1, cache1, "relu")

        grads["dw1"] = dw1
        grads["db1"] = db1
        grads["dw2"] = dw2
        grads["db2"] = db2

        params = updateParams(params, grads, learningRate)

        
        if printCost and i%100 == 0: 
            costs.append(cost)
            print(f"Cost for {i}th iteration: {cost}")
    
    plt.plot(np.squeeze(costs))
    plt.ylabel("Cost")
    plt.xlabel("Iteration (per hundred)")
    plt.title(f"Learning Rate {learningRate}")
    plt.show

    return params


In [30]:
params = twoLayerModel(x=trainX, y=trainDatasetY, layerDims=layerDims, numIterations=2500, printCost=True)

Cost for 0th iteration: 0.829942437000611


KeyboardInterrupt: 